In [127]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

In [15]:
# Load info_01.csv
df1 = pd.read_csv("ds\\info_01.csv")
def check_diff(arr1,arr2):
    arr3 = arr1 - arr2
    return sum(arr3)

In [16]:
df1 = pd.read_csv("ds\\info_01.csv").drop_duplicates()

In [17]:
#Apparently id and id2 are the same lets check with a quick aggregation
check_diff(df1["id"],df1["id2"])

0

In [18]:
df2 = pd.read_csv("ds\\info_02.csv")

In [19]:
# I do have the same problem as in dataset 1. The id and id2 seems to be the same. Let me do a quick verification.
check_diff(df2["id"],df2["id2"])

0

Given the information above it was found that both datasets can use only one attribute related to the id. Thus, I am going to leave only id. 

In [20]:
df1 = df1.drop(columns=["id2"])
df2 = df2.drop(columns=["id2"])

In [21]:
df2 = df2.drop_duplicates()
print("amount of good registers in info_01 ",len(df1))
print("amount of good registers in info_02 ",len(df2))

amount of good registers in info_01  8143
amount of good registers in info_02  9143


In [22]:
df_final = df2.merge(df1, how='left', on=['id'])

# Feature Engineering

In [26]:
def get_date(string):
    return pd.to_datetime(string)
def get_day(date):
    return date.day
def get_hour(date):
    return date.hour
def get_min(date):
    return date.minute
def get_second(date):
    return date.second
#df_final = df_final.drop(columns=["id","v3","c1"])

In [29]:
df_final["v3"] = list(map(get_date,df_final["v3"]))
df_final["day"] = list(map(get_day, df_final["v3"]))
df_final["hour"] = list(map(get_hour, df_final["v3"]))
df_final["minute"] = list(map(get_min, df_final["v3"]))
df_final["second"] = list(map(get_second, df_final["v3"]))
df_final["v7"] = df_final["v4"] - df_final["v1"]

In [31]:
df_final = df_final.drop(columns=["c1","id","v3"])

In [32]:
df_final 

,v1,v2,v4,v5,v6,day,hour,minute,second,v7
0,426.0,23.2,721.2,27.3,0.004793,4.0,17.0,51.0,0.0,295.2
1,429.5,23.1,714.0,27.3,0.004783,4.0,17.0,51.0,59.0,284.5
2,426.0,23.1,713.5,27.2,0.004779,4.0,17.0,53.0,0.0,287.5
3,426.0,23.1,708.2,27.2,0.004772,4.0,17.0,54.0,0.0,282.2
4,426.0,23.1,704.5,27.2,0.004757,4.0,17.0,55.0,0.0,278.5
5,419.0,23.1,701.0,27.2,0.004757,4.0,17.0,55.0,59.0,282.0
6,419.0,23.1,701.7,27.2,0.004757,4.0,17.0,57.0,0.0,282.7
7,419.0,23.1,699.0,27.2,0.004757,4.0,17.0,57.0,59.0,280.0
8,419.0,23.1,689.3,27.2,0.004757,4.0,17.0,58.0,59.0,270.3
9,419.0,23.1,688.0,27.2,0.004745,4.0,18.0,0.0,0.0,269.0


In [37]:
sc = MinMaxScaler()
x_scale = pd.DataFrame(sc.fit_transform(df_final))
x_scale = x_scale.fillna(.999999)

In [148]:
kmeans = KMeans(n_clusters=10, n_init=20, algorithm='elkan', random_state=0).fit(x_scale)
labels = kmeans.labels_

In [149]:
len(list(kmeans.labels_))

9143

# Consistency of the clusters 

scale 0.0 - 1.0 

In [150]:
silhouette_score(x_scale, labels, metric='euclidean')

0.31050931584421576

# Reason why I choose this clustering algorithm

It was the algorithm that it gave me best results given the type of data.